## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
columns_to_drop = ['EIN', 'NAME']
application_df.drop(columns=columns_to_drop, inplace=True)

In [3]:
# Determine the number of unique values in each column.
unique_values_per_column = application_df.nunique()

# Display the result
print(unique_values_per_column)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the result
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Choose a cutoff value (adjust as needed)
cutoff_value = 100  

# Identify application types with counts less than the cutoff value
infrequent_application_types = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff_value].index.tolist()

# Create a list of application types to be replaced
application_types_to_replace = infrequent_application_types

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the result
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [1]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter value counts where count is greater than 1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(classification_counts_greater_than_1)


NameError: name 'application_df' is not defined

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 100 

# Identify classifications with counts less than the cutoff value
infrequent_classifications = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value].index.tolist()

# Create a list of classifications to be replaced
classifications_to_replace = infrequent_classifications

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
# Specify the columns to one-hot encode
columns_to_encode = ['APPLICATION_TYPE', 'CLASSIFICATION']

# Use pd.get_dummies to perform one-hot encoding
application_df_encoded = pd.get_dummies(application_df, columns=columns_to_encode)

# Display the result
print(application_df_encoded.head())

        AFFILIATION      USE_CASE  ORGANIZATION  STATUS     INCOME_AMT  \
0       Independent    ProductDev   Association       1              0   
1       Independent  Preservation  Co-operative       1         1-9999   
2  CompanySponsored    ProductDev   Association       1              0   
3  CompanySponsored  Preservation         Trust       1    10000-24999   
4       Independent     Heathcare         Trust       1  100000-499999   

  SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0                      N     5000              1                       0   
1                      N   108590              1                       0   
2                      N     5000              0                       0   
3                      N     6692              1                       0   
4                      N   142590              1                       0   

   APPLICATION_TYPE_T10  ...  CLASSIFICATION_C1270  CLASSIFICATION_C1700  \
0                     

In [9]:
# Split our preprocessed data into our features and target arrays
# Split the preprocessed data into a training and testing dataset
# Specify the features (X) and target (y)
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting arrays
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (27439, 29)
X_test shape: (6860, 29)
y_train shape: (27439,)
y_test shape: (6860,)


In [12]:
# Identify categorical columns
categorical_columns = application_df.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)

# Check data types after encoding
print(application_df_encoded.dtypes)

# Separate features (X) and target (y)
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit and transform the data
X_scaled = scaler.fit_transform(X)

# Split the scaled data into training and testing sets
X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting arrays
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

STATUS                          int64
ASK_AMT                         int64
IS_SUCCESSFUL                   int64
APPLICATION_TYPE_Other          uint8
APPLICATION_TYPE_T10            uint8
APPLICATION_TYPE_T19            uint8
APPLICATION_TYPE_T3             uint8
APPLICATION_TYPE_T4             uint8
APPLICATION_TYPE_T5             uint8
APPLICATION_TYPE_T6             uint8
APPLICATION_TYPE_T7             uint8
APPLICATION_TYPE_T8             uint8
APPLICATION_TYPE_T9             uint8
AFFILIATION_CompanySponsored    uint8
AFFILIATION_Family/Parent       uint8
AFFILIATION_Independent         uint8
AFFILIATION_National            uint8
AFFILIATION_Other               uint8
AFFILIATION_Regional            uint8
CLASSIFICATION_C1000            uint8
CLASSIFICATION_C1200            uint8
CLASSIFICATION_C1270            uint8
CLASSIFICATION_C1700            uint8
CLASSIFICATION_C2000            uint8
CLASSIFICATION_C2100            uint8
CLASSIFICATION_C2700            uint8
CLASSIFICATI

## Compile, Train and Evaluate the Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]

# Create a Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer with 128 nodes and 'relu' activation function
nn.add(Dense(units=128, input_dim=input_features, activation='relu'))

# Second hidden layer with 64 nodes and 'relu' activation function
nn.add(Dense(units=64, activation='relu'))

# Output layer with a single node and 'sigmoid' activation function for binary classification
nn.add(Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               6528      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 14849 (58.00 KB)
Trainable params: 14849 (58.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/50


858/858 [==============================] - 2s 2ms/step - loss: 0.5657 - accuracy: 0.7238 - val_loss: 0.5620 - val_accuracy: 0.7210
Epoch 2/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5521 - accuracy: 0.7298 - val_loss: 0.5571 - val_accuracy: 0.7267
Epoch 3/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7309 - val_loss: 0.5575 - val_accuracy: 0.7281
Epoch 4/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7329 - val_loss: 0.5549 - val_accuracy: 0.7296
Epoch 5/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5458 - accuracy: 0.7332 - val_loss: 0.5550 - val_accuracy: 0.7273
Epoch 6/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5452 - accuracy: 0.7335 - val_loss: 0.5566 - val_accuracy: 0.7315
Epoch 7/50
858/858 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 0.7341 - val_loss: 0.5532 - val_accuracy: 0.7276
Epoc

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5600 - accuracy: 0.7262 - 197ms/epoch - 917us/step
Loss: 0.5600032806396484, Accuracy: 0.7262390851974487


In [22]:
# Export our model to HDF5 file
model_filename = 'AlphabetSoupCharity_Optimization.keras'

# Save the model to the native Keras format
nn.save(model_filename)
